In [59]:
import pandas as pd
import psycopg2

In [61]:
#Get DB credentials
from dotenv import load_dotenv
import os

from dotenv import load_dotenv
from pathlib import Path
import os

dotenv_path = Path('db_credentials.env')
load_dotenv(dotenv_path=dotenv_path)

DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_DATABASE')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DATABRICKS_TOKEN = os.getenv('DATABRICKS_TOKEN')
DATABRICKS_INSTANCE = os.getenv('DATABRICKS_INSTANCE')

In [99]:
try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    
    cursor = connection.cursor()
    
    cursor.execute("""
        DROP VIEW IF EXISTS silver_consolidated_airline_reviews
    """)
    
    cursor.execute("""
        CREATE VIEW silver_consolidated_airline_reviews AS
        SELECT
        	CONCAT('K', "Id") AS airline_review_id,
            "Airline Name" AS airline_name,
        	"Overall_Rating" AS rating_overall,
        	"Review_Title" AS review_title,
        	"Review Date" AS review_date,
        	"Verified" AS verified,
        	"Review" AS review_text,
        	"Aircraft" AS aircraft,
        	"Type Of Traveller" AS type_of_traveller,
        	"Seat Type" AS seat_type,
        	"Route" AS route,
        	"Date Flown" AS flight_date,
        	"Seat Comfort" AS rating_seat_comfort,
        	"Cabin Staff Service" AS rating_cabin_staff_service,
        	"Food & Beverages" AS rating_food_and_beverages,
        	"Ground Service" AS rating_ground_service,
        	"Inflight Entertainment" AS rating_inflight_entertainment,
        	"Wifi & Connectivity" AS rating_wifi_and_connectivity,
        	"Value For Money" AS rating_value_for_money,
        	"Recommended" AS recommended,
            "is_lost_luggage_flag",
            CAST(NULL AS bool) is_low_cost_airline,
            "created_by",
            "created_date",
            "lastmodified_by",
            "lastmodified_date"
        FROM silver_airline_quality_reviews AS K

        UNION ALL

        SELECT
        	R.post_id AS airline_review_id,
            AC."Name" AS airline_name,
            --CONCAT(AC."Name", ' (', L.responsible_airline, ')') AS airline_name,
        	NULL AS rating_overall,
        	R.title AS review_title,
        	R.created_utc AS review_date,
        	NULL AS verified,
        	R.selftext AS review_text,
        	NULL AS aircraft,
        	NULL AS type_of_traveller,
        	NULL AS seat_type,
        	CASE
                WHEN L.mentioned_airlines_routes = '{}' THEN null
                ELSE
                    NULLIF(
                    REPLACE(
                    REPLACE(
                    REPLACE(
                    REPLACE(
                    REPLACE(
                    REPLACE(
                    REPLACE( 
                        REPLACE(L.mentioned_airlines_routes, '},{', ' to ')
                        , ',', ' to ')
                        , '{{', '')
                        , '}}', '')
                        , '{', '')
                        , '}', '')
                        , 'unknown to ', '')
                        , 'unknown', '')
                        ,'')
            END AS route,
        	NULL AS flight_date,
        	NULL AS rating_seat_comfort,
        	NULL AS rating_cabin_staff_service,
        	NULL AS rating_food_and_beverages,
        	NULL AS rating_ground_service,
        	NULL AS rating_inflight_entertainment,
        	NULL AS rating_wifi_and_connectivity,
        	NULL AS rating_value_for_money,
        	NULL AS recommended,
            CASE WHEN L.luggage_issue = 1 THEN true ELSE false END AS is_lost_luggage_flag,
            CAST(NULL AS bool) is_low_cost_airline,
            'From_Reddit' AS created_by,
            R.created_utc AS created_date,
            'From_Reddit' AS lastmodified_by,
            R.created_utc AS lastmodified_date
        FROM bronze_reddit_reviews AS R
        INNER JOIN labeling_values AS L ON L.post_id = R.post_id
        INNER JOIN bronze_airline_codes AS AC ON AC."ICAO" = L.responsible_airline
        WHERE L.responsible_airline != 'unknown'
        AND L.aviation_related = 1
    """)
    connection.commit()
    print("View 'silver_consolidated_airline_reviews' created successfully.")

except (Exception, psycopg2.Error) as error:
    print("Error during drop-create view operation:", error)
finally:
    if connection:
        cursor.close()
        connection.close()

View 'silver_consolidated_airline_reviews' created successfully.


In [83]:
results = None
colnames = None

try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )

    cursor = connection.cursor()
    
    query = 'SELECT * FROM "silver_consolidated_airline_reviews"'
     
    cursor.execute(query)
     
    results = cursor.fetchall()

    # Get the column names
    colnames = [desc[0] for desc in cursor.description]

    # Convert the results to a pandas DataFrame
    df = pd.DataFrame(results, columns=colnames)

except (Exception, psycopg2.Error) as error:
    print("Error during select operation:", error)
finally:
    if connection:
        cursor.close()
        connection.close()

print(df.head())
print(df.columns)
print(len(df))

                                   airline_review_id   airline_name  \
0  K5cb0f79de5c9e67eb8b30829f79c4954df18ccb36ac93...    AB Aviation   
1  K8e6c32573411848ebb03db8611ba048ed404efb10e302...    AB Aviation   
2  K87eb30af6644384e7f6b999af1496654808d0b20fdbf4...    AB Aviation   
3  K7f7f715210dce748407158608b5d19354598b50d5019a...  Adria Airways   
4  K7da7e8cd22cf9233f1427953365c03263d0951b930f3d...  Adria Airways   

   rating_overall                           review_title review_date verified  \
0             9.0                "pretty decent airline"  2019-11-11     True   
1             1.0                   "Not a good airline"  2019-06-25     True   
2             1.0         "flight was fortunately short"  2019-06-25     True   
3             1.0    "I will never fly again with Adria"  2019-09-28    False   
4             1.0  "it ruined our last days of holidays"  2019-09-24     True   

                                         review_text       aircraft  \
0    Moroni to 

#### Upload Dataframe to Databricks volume

In [108]:
# Only upload to Databricks Airline Quality Reviews
#df.to_csv('data/silver_consolidated_airline_reviews.csv')

In [110]:
#import requests

In [117]:
'''
#Using Databricks Upload a file API
#https://docs.databricks.com/api/workspace/files/upload
databricks_instance = DATABRICKS_INSTANCE
local_path = 'data/silver_consolidated_airline_reviews.csv'
remote_path = '/Volumes/workspace/data608/data608_volume/silver_consolidated_airline_reviews.csv'
headers = {'Authorization' : f"Bearer {DATABRICKS_TOKEN}"}
#print(headers)

with open(local_path, 'rb') as f:
    r = requests.put(
      f"https://{databricks_instance}/api/2.0/fs/files{remote_path}?overwrite=true",
      headers=headers,
      data=f)
    r.raise_for_status()

print(f"Uploaded '{local_path}' to '{remote_path}' on Databricks workspace {databricks_instance}")
'''

Uploaded 'data/silver.consolidated_airline_reviews.csv' to '/Volumes/workspace/data608/data608_volume/silver_consolidated_airline_reviews.csv' on Databricks workspace dbc-daf0509f-1d8b.cloud.databricks.com
